In [14]:
import numpy as np
from numba import jit
from scipy import linalg as la

def pymatpow(X, power):
    prod = X.copy()
    temparr = np.empty_like(X[0])
    size = X.shape[0]
    for n in xrange(1,power):
        for i in xrange(size):
            for j in xrange(size):
                tot = 0.
                for k in xrange(size):
                    tot += prod[i,k] * X[k,j]
                temparr[j] = tot
            prod[i]  = temparr
    return prod

@jit
def numbamatpow(X, power):
    prod = X.copy()
    temparr = np.empty_like(X[0])
    size = X.shape[0]
    for n in xrange(1,power):
        for i in xrange(size):
            for j in xrange(size):
                tot = 0.
                for k in xrange(size):
                    tot += prod[i,k] * X[k,j]
                temparr[j] = tot
            prod[i]  = temparr
    return prod

def npdot(X,power):
    temp = X.copy()
    for i in xrange(1,power):
        temp = np.dot(temp,X)
    return temp

def pytridiag(a,b,c,x):
    print "called pytridiag"
    n = x.size
    temp = 0.
    c[0] /= b[0]
    x[0] /= b[0]
    for i in xrange(n-2):
        temp = 1./(b[i+1] - a[i]*c[i])
        c[i+1] *= temp
        x[i+1] = temp*(x[i+1] - a[i]*x[i])
    x[n-1] = (x[n-1] - a[n-2]*x[n-2]) / (b[n-1] - a[n-2]*c[n-2])
    for i in xrange(n-2,-1,-1):
        x[i] = x[i] - c[i]*x[i+1]
    return x

@jit
def numbatridiag(a,b,c,x):
    n = x.size
    temp = 0.
    c[0] /= b[0]
    x[0] /= b[0]
    for i in xrange(n-2):
        temp = 1./(b[i+1] - a[i]*c[i])
        c[i+1] *= temp
        x[i+1] = temp*(x[i+1] - a[i]*x[i])
    x[n-1] = (x[n-1] - a[n-2]*x[n-2]) / (b[n-1] - a[n-2]*c[n-2])
    for i in xrange(n-2,-1,-1):
        x[i] = x[i] - c[i]*x[i+1]
    return x
    
def prob4():
    # Compare pymatpow, numbamatpow, and np.dot
    pass
    
    X = np.random.rand(10,10)
    n = 5

    print "Pymatpow:"
    %timeit pymatpow(X,n)
    print "Numbamatpow:"
    %timeit numbamatpow(X,n)
    print "Using np.dot:"
    %timeit npdot(X,n)
    print "Obviously np.dot() is the fastest, but numba did give us some speedup."

def prob5():
    # Compare pytridiag, numbatridiag, scipy.linalg.solve()
    n = 10
    a = np.random.random_integers(-9,9,n).astype('float')
    b = np.random.random_integers(-9,9,n).astype('float')
    c = np.random.random_integers(-9,9,n).astype('float')
    A = np.zeros((n,n))
    np.fill_diagonal(A,b)
    np.fill_diagonal(A[1:,:-1],a)
    np.fill_diagonal(A[:-1,1:],c)
    x = np.random.random_integers(-9,9,n).astype('float')
    
    
    a = np.random.rand(n-1)
    b = np.random.rand(n)
    c = np.random.rand(n-1)
    A = np.diag(a,-1) + np.diag(b) + np.diag(c,1)
    x = np.random.rand(n)
    
    print "scipy.linalg.solve timing:"
    %timeit la.solve(A,x)
    print "numbatridiag timing:"
    %timeit numbatridiag(a,b,c,x)
    print "pytridiag timing:"
    %timeit pytridiag(a,b,c,x)

[-0.42740689  1.32248043  2.38794644 -1.23056973  0.8780114   3.23650738
 -2.43904071 -2.31016141  1.30553625  0.1147074 ]
[-0.42740689  1.32248043  2.38794644 -1.23056973  0.8780114   3.23650738
 -2.43904071 -2.31016141  1.30553625  0.1147074 ]
called pytridiag
[ 1.89098731 -0.72261216 -2.16172598  0.23579704 -8.5074734   7.15840964
 -1.68652086  9.50888815 -0.79567327  0.63166143]
scipy.linalg.solve timing:


NameError: global name 'A' is not defined